# Motor 
## Motor current of SGMADA 27063800
* Door weight : 336g
* Vcc= 6V
* Motor overload current : > 50mA
* Motor current during lifting : 37-43mA
* Motor current during no load : 19.5 - 21mA
* Motor current during lowering : 13.5 - 15mA

## Motor driver
Motor driver will be directly fed from the battery.  The back-EMF will be absorbed by the battery.

### TI DRV8231ADDAR
#### Current sense
* Select Rsense so that 30mA is in the middle of the ADC range.
* Rsense = 15Kohm
* Provide a 10nF capacitor parallel to Rsense (prevents premature trigger of current regulation)
* Vref = 2.25V 
  * input current to Vref not defined
  * Problem : power consumption of Vref when in sleep mode:
  * Solution : diode OR-ing of IN1 and IN2 charges a capacitor to Vx.  A TLV431, powered from Vx then provides Vref.

In [4]:
Imotor = 30e-3 # A
Vsense = 0.65 # V
Ai = 1500e-6 # A/A
Rsense = Vsense / (Imotor * Ai)

Rsense = 15e3
Itrip = 0.1 # A
Vref = Itrip * Ai * Rsense # V, must be between 0 and 3.6V

#### Low power sleep
* Input pins have 100Kohm pull-down
* Bringing  both inputs low puts the device in sleep mode
* Low power sleep current : 1µA (Ivmq)

#### Current regulation during startup and stall
* UVLO = 4.5V
* Add 47µF || 100nF on Vm
* Ramping up PWM might be a way to avoid tripping current regulation during startup.

### Toshiba TB67H450AFNG,EL
* Motor current flows through Rsense.

In [5]:
Itrip = 0.1
Rsense = 1
Ai = 10
Vref = Itrip * Ai * Rsense

When Rsense is 1ohm, MCU ADC attenuation should be ADC_ATTEN_DB_2_5 (max 1.05V input)

# MCU
ESP32-C3
* only 13 GPIO, but that should be enough for this application
* uses only half the current of ESP32-S3 in similar conditions (it's only single core)
* smaller than ESP32-S3 module
* 40% cheaper than ESP32-S3

## Programming & debug
* No source resistor needed on TXD, as the module already has R2, a 499(ohm?) resistor built in.
* Level-shifting circuitry to protect device from IO-programmer voltage, which might be 5V.
* ESP-Prog will be used for programming and debug.
* Programming & reset buttons are on the ESP-Prog.  There's no need for these buttons on the device.

## Motor 
### Motor pins
* 2 GPIOs needed.
* Pull-down in the motor controller, so there should be no pull-up on the MCU.  Reset state of the MCU pin can be found in [ESP32-C3 Technical reference manual](https://www.espressif.com/sites/default/files/documentation/esp32-c3_technical_reference_manual_en.pdf), §5.12 IO MUX Functions List.
* The glitches at power up are acceptable for this product.
* GPIO7 and GPIO10 will be used.

### Current sense connection
* Use ADC1, it's factory calibrated (ADC2 isn't)
* Lowest relative error in ATTEN2 range (up to 1.3V input) : [ADC_ATTEN_DB_6](https://docs.espressif.com/projects/arduino-esp32/en/latest/api/adc.html)
* Put a series resistor to limit the input current in case over-current events.
* GPIO3 is used.  Alternative pins is GPIO4.  GPIO2 is not suitable because of its required pull-up.

## RTC
The cheapest option will be to use the MCU's built-in RTC.  The problem with this is the large power consumption, because the power supply pins of the RTC are connected to the power supply pins of the main MCU.  As such, the main MCU is always powered, which wastes power.

* 32kHz crystal needed (is on ADC1.CH0 and ADC1.CH1).  RTC-counter can have other sources, but these are not accurate enough.
* Occupies GPIO0 & GPIO1
* Power consumption in deep-sleep mode is 5µA.  Only RTC-Timer and RTC-Memory are active.

## Buttons
There are no three pins available for reading three buttons.  An extra requirement is that pushing a button should wake up the MCU in case it's sleeping.

This problem is solved by using a resistor divider circuit and connecting each button to it.  Capacitors are added to generate a falling edge (and wake up the MCU) at the start of the button press.

Only ADC-pins powered by the VDD3P3_RTC are suitable for this purpose :  GPIO2, GPIO3, GPIO4 (GPIO5 is connected to ADC2, to be avoided)

By using GPIO2, we can repurpose the mandatory bootstrapping pull-up.

An alternative solution would be to use an IO-expander, but that would require an extra pin for the IRQ anyway.

## Display interface
I²C is used.  It requires only two GPIOs.  By using GPIO8 and GPIO9, we can repurpose the pull-up resistors which are already there for bootstrapping.

# Display
## Display type
The original ChickenGuard uses a transmissive 16x2 character display.  Unfortunately the contrast was very poor.
Transmissive technology means that the back light needs to be on for the characters to be visible.
A transflective display might be a better option as it's not necessary to turn on the backlight during daylight.

The display module dimensions are 80x36mm, which is quite standard.  Unfortunately, the metal frame is only 24.2x71.2mm, which is smaller than the display modules on Digikey (OrientDisplay AMC1602AR and Gi Far Technology Co Ltd. GFC1602AK).  It looks however identical to the €2 LCD1602 offerings on AliExpress.

* Transmissive source from [DFRobot](https://www.dfrobot.com/product-451.html) : datasheet probably shows wrong size of metal frame.
* Transflective source from [DFRobot](https://www.dfrobot.com/product-456.html) : datasheet probably shows wrong size of metal frame. 

By lack of a better display, the display of the original Chickenguard will be salvaged.

## IO expander
* TI TCA9534PWR
* NXP PCA9534PW,118

## I2C level translator
* Discrete implementation using two NMOS and a couple of resistors.

## Digital potentiometer
Instead of an analog potentiometer, which requires manual adjustment and can not easily be changed in the field, a digital potentiometer will be used:
* Microchip MCP40D18-xxxE/LT
* Renesas ISL90727
* TI TPL0401A

# Power
## Alkaline AA-battery
* Looses 3% of its capacity per year.
* Capacity of AA is around 2000mAh.

In [6]:
initial_charge = 2 # Ah
lost_charge = 0.03 * initial_charge # Ah
lost_charge_Coulomb = lost_charge * 3600 # Coulomb
leakage_current = lost_charge_Coulomb / (3600 * 24 * 365) # A

This corresponds to a leakage current of 7µA at the start.
We should try to keep power consumption in sleep mode smaller than the battery self-discharge.  There's not much point in doing better than that.

## Power path
As the battery voltage can be more than a diode drop higher than the USB-voltage, the default power path circuit with a PMOS and a schottky diode can no longer be used.  The body diode of the PMOS would always be conducting, even if USB was present, so the battery would be drained even if USB is present.

A new power path circuit has been developed to overcome this problem.

## LD0 5V, low quiescent current
The maximum input voltage of the 3V3 LDO is only 6.5V, which is rather limited.  To protect it from higher voltages, a simple discrete pre-regulator has been added.

## LDO 5V, high current
### Integrated solution
SMD (no QFN or BGA), Iout >=500mA, Input max. >= 10V, 5V out, 0.33V dropout at 500mA, < €2/pce
Only the **R1500H050B-T1-FE** (900mW) comes out as solution (€1.44/pce).
There's already an integrated enable pin to turn the supply ON/OFF.

### Discrete solution
A simple discrete solution can be made cheaper, but then there's no over-current protection.

## LDO 3V3, high current
Using the XC6220B331MR-G.  Alternative part is the RT9080-33GJ5.
Their quiescent current is <5µA, while their output current > 500mA.
The limited input voltage (<6.5V) is a drawback.

## Battery voltage sense
No distinction is being made between USB-power and battery power.  Why should we?  If the power is too low, no matter what source, the device will work incorrectly.
When battery voltage is too low, batteries need to be changed.  When USB-voltage is too low, it's likely that there's an issue with the USB-cable (too high internal resistance).
In both cases, the user should be informed about the issue.

A resistive divider using relatively small resistors (10K range) yields the best accuracy.  Unfortunately the current drain during sleep mode is unacceptable.  By adding an NMOS, which is only turned on when the 5V is present, this issue is solved.  The NMOS in the circuit should have a low Vgsth and a low RdsON.

# Connectors
* USB-C for power and data
* battery : JST PH : 3 pin
* motor : JST PH : 2 pin

# LED
There's only one pin left.  A WS2812B RGB-LED could be used, as on the ESP32-C3 development kit.
Another option would be to control an RGB-LED with an I²C IO-expander (that could also be used to read the buttons).  This extra cost doesn't justify the extra functionality.
Let's reduce this to the use of a single color RED LED that flashes when the power supply voltage is too low.

# Web interface
## Button on webpage controls GPIO
Used technologies : websockets, javascript, html, css, JSON

* [ESP32 Remote Control with WebSocket](https://m1cr0lab-esp32.github.io/remote-control-with-websocket/)
* [ESP32 WebSocket Server: Control Outputs (Arduino IDE)](https://randomnerdtutorials.com/esp32-websocket-server-arduino/)

## Geo-location
[HTML Geolocation API](https://www.w3schools.com/html/html5_geolocation.asp)

## Time
[HTML element time](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/input/time)

# Libraries
* [Solar Calculator](https://github.com/jpb10/SolarCalculator)
* [ArduinoJson](https://github.com/bblanchon/ArduinoJson)

### Visualize time
![sunset](https://icon-library.com/images/sunset-icon/sunset-icon-17.jpg)

* [Sunrise icon](https://icon-library.com/icon/sunrise-icon-png-16.html)

![Sunrise](https://icon-library.com/images/sunrise-icon-png/sunrise-icon-png-16.jpg)

* [Sunset icon](https://icon-library.com/icon/sunset-icon-17.html)
